In [1]:
import networkx as nx
import csv
import numpy as np
%matplotlib inline 
from matplotlib import pyplot as plt
import os
import pandas as pd


# Exploration notebook

**TODO: see https://www.notion.so/Networks-ML-project-1230cf1ae7ec497eb5845daa0442e993 for ideas of exploration**. I think it is good to start with basic things, like we've done in the first homeworks, before looking into more advanced stuff like interactive animations. I really believe it is more important for the graders to see that we understood the concepts introduced earlier in the course, like the different random graph models, and then if we have time, we can make some more fancy/advanced things.

In [ ]:
def load_graph(city, mode):
    edges = pd.read_csv(f"all_cities/{city}/network_{mode}.csv", sep=";")
    nodes = pd.read_csv(f"all_cities/{city}/network_nodes.csv", sep=";")
    
    graph = nx.DiGraph()
    for idx, row in nodes.iterrows():
        graph.add_node(
            row.stop_I, 
            lat=row.lat, 
            lon=row.lon, 
            name=row.name, 
            pos=(row.lon, row.lat)
        )

    for idx, row in edges.iterrows():
        graph.add_edge(
            row.from_stop_I, 
            row.to_stop_I, 
            d=row.d, 
            duration_avg=row.duration_avg, 
            n_vehicles=row.n_vehicles
        )

        
    return graph
    

In [24]:
def draw_graph(graph, ax, node_size=0.5, no_edge_alpha=0.5):
    # Plot disconnected edges lighter
    degrees = nx.degree(graph)
    node_with_edges = []
    node_without_edges = []

    for node in graph.nodes():
        if degrees[node] > 0:
            node_with_edges.append(node)
        else:
            node_without_edges.append(node)

    # Split graph based on non-zero-degree
    no_edge_subgraph = graph.subgraph(node_without_edges)
    with_edges_subgraph = graph.subgraph(node_with_edges)
    # No edges
    nx.draw_networkx_nodes(
        no_edge_subgraph,
        pos=nx.get_node_attributes(no_edge_subgraph, "pos"),
        node_size=node_size,
        ax=ax,
        alpha=no_edge_alpha,
        node_color='lightgreen',
    )
    # With edges
    nx.draw_networkx(
        with_edges_subgraph,
        pos=nx.get_node_attributes(with_edges_subgraph, "pos"),
        with_labels=False,
        node_size=node_size,
        ax=ax,
        alpha=0.7,
        arrows=False,
        edge_color='orange',
    )
    


In [17]:
def city_has_mode(city, mode):
    return os.path.exists(f"all_cities/{city}/network_{mode}.csv")

In [18]:
all_cities = [city for city in os.listdir("all_cities") if os.path.isdir(f"all_cities/{city}")]
all_modes = ['walk', 'tram', 'subway', 'rail', 'bus', 'ferry', 'cablecar', 'gondola', 'funicular', 'combined']


# Count how many cities have each transportation mode
mode_counts = []
for mode in all_modes:
    cities = [c for c in all_cities if os.path.exists(f"all_cities/{c}/network_{mode}.csv")]
    mode_counts.append((mode, len(cities)))

mode_counts.sort(key=lambda x: x[1], reverse=True)

for mode, counts in mode_counts:
    print(mode, counts)
    

walk 25
bus 25
combined 25
tram 14
rail 12
subway 8
ferry 8
cablecar 1
gondola 0
funicular 0


In [26]:
mode = 'bus'
os.makedirs("plots", exist_ok=True)
for city in all_cities:
    if not city_has_mode(city, mode):
        continue
    fig, ax = plt.subplots()
    graph = load_graph(city, mode)
    draw_graph(graph, ax)
    fig.suptitle(city.capitalize())
    fig.savefig(f"plots/{city}-{mode}.png", dpi=300)
    plt.close()

Baseline ETA/Congestion: shortest path + node/edge centrality as proxy for congestion; tune baseline on that feature + others to see

TODO: 
* Plot histogram of edge congestion (avg duration on edges)
* Plot percentage of edges in each network (tram vs subway vs bus, etc)
* Plot degree distribution; check small world property, if close to erdos etc


In [ ]:
nx.conne

In [94]:
def compute_statistics(graph):
    """
    Aggregated statistic over graphs. Try kmeans on top to see if can cluster graphs into visually similar ones
    """
    stats = {}
    stats['avg_clustering'] = nx.average_clustering(graph)
    
    # Best result in PSET 2; argue that's why we use it in report
    eigvec_centrality = nx.eigenvector_centrality(graph, max_iter=1000)
    eigvec_values = [v for k, v in eigvec_centrality.items()]
    stats['avg_eig_centr'] = np.mean(eigvec_values)
    stats['std_eig_centr'] = np.std(eigvec_values)
    # TODO: very long to compute
    #betw_centrality = nx.betweenness_centrality(graph)
    #betw_values = [v for k, v in betw_centrality.items()]
    #stats['avg_betw_centr'] = np.mean(betw_values)
    #stats['std_betw_centr'] = np.std(betw_values)

    #close_centrality = nx.closeness_centrality(graph)
    #close_centr_values = [v for k, v in close_centrality.items()]
    #stats['avg_close_centr'] = np.mean(close_centr_values)
    #stats['std_close_centr'] = np.std(close_centr_values)

    degree_distr = nx.degree(graph)
    degree_distr_values = [v for k, v in degree_distr]
    stats['avg_degree_distr'] = np.mean(degree_distr_values)
    stats['std_degree_distr'] = np.std(degree_distr_values)
    sec_moment = [v ** 2 for v in degree_distr_values]
    stats['avg_sec_moment'] = np.mean(sec_moment)
    stats['std_sec_moment'] = np.std(sec_moment)

    ccs = nx.connected_components(nx.to_undirected(graph))
    largest_cc_idxs = max(ccs, key=len)
    largest_cc = nx.subgraph(graph, largest_cc_idxs)
    # stats['avg_path_lengths'] = nx.average_shortest_path_length(largest_cc)
    stats['frac_node_in_largest_cc'] = len(largest_cc_idxs) / nx.number_of_nodes(graph)
    return stats
    

In [97]:
def degree_distribution(graph):
    pass